In [56]:
import numpy as np
import tensorflow as tf
import keras
from time import time

# Load the Shakespeare text file / Setup Tokenizer

In [57]:
path_to_file = keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)
print(path_to_file)

with open(path_to_file) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

shakespeare_tensor = tf.constant([shakespeare_text])

/tmp/.keras/datasets/shakespeare.txt
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [58]:
set(shakespeare_text)

{'\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [59]:
len(set(shakespeare_text.lower().strip()))

39

In [60]:
vocab = list(set(shakespeare_text.lower().strip()))

In [61]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(shakespeare_text.lower())
config = tokenizer.get_config()
tokenizer.texts_to_sequences(["hello", "world"])

[[7, 2, 12, 12, 4], [17, 4, 9, 12, 13]]

In [62]:
n_tokens = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [63]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))
print(f"{encoded[:10]}")

[20  6  9  8  3  1 19  6  3  6]


# Create Dataset

In [64]:
train_size = dataset_size * 90 // 100
print(f"train_size = {train_size:,}")

train_size = 1,003,854


In [65]:
dataset = tf.data.Dataset.from_tensor_slices(encoded)
#dataset = dataset.repeat()
n_steps = 50
dataset = dataset.window(n_steps + 1, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window_ds: window_ds.batch(n_steps + 1))

In [66]:
for x in dataset.take(2):
    print(x)

tf.Tensor(
[20  6  9  8  3  1 19  6  3  6 36  2 10 24 11 22  2 20  4  9  2  1 17  2
  1 23  9  4 19  2  2 13  1  5 10 16  1 20 14  9  3  7  2  9 18  1  7  2
  5  9  1], shape=(51,), dtype=int64)
tf.Tensor(
[ 6  9  8  3  1 19  6  3  6 36  2 10 24 11 22  2 20  4  9  2  1 17  2  1
 23  9  4 19  2  2 13  1  5 10 16  1 20 14  9  3  7  2  9 18  1  7  2  5
  9  1 15], shape=(51,), dtype=int64)


In [67]:
dataset = dataset.batch(128)

In [68]:
for x in dataset.take(1):
    print(x.shape)

(128, 51)


In [69]:
dataset = dataset.map(lambda window: (window[:, :-1], window[:, 1:]))

In [70]:
dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)

In [71]:
for x, y in dataset.take(1):
    print(f"x = {x.shape}")
    print(f"y = {y.shape}")

x = (128, 50)
y = (128, 50)


2025-04-11 06:42:14.807430: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Create Models

In [79]:
m1 = keras.Sequential(
    [
        keras.layers.Input(shape=(None, n_tokens)),
        keras.layers.GRU(128, return_sequences=True),
        keras.layers.Dense(n_tokens, activation="softmax")
    ],
)
m1.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m1.summary()
model = m1

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_3 (GRU)                     │ (None, None, 128)      │        64,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,927 (273.15 KB)

 Trainable params: 69,927 (273.15 KB)

 Non-trainable params: 0 (0.00 B)

In [80]:
text = "to be "
tokens = tokenizer.texts_to_sequences([text])
inp_tf = tf.constant(tokens)
inp_tf = tf.one_hot(inp_tf, depth=n_tokens)
print(inp_tf.shape)
y = model.predict(inp_tf)
print(y.shape)
y = tf.argmax(y[0,-1])

(1, 6, 39)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
(1, 6, 39)


In [82]:
early_stopping = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)
history = m1.fit(dataset, epochs=10, steps_per_epoch=train_size // 128, callbacks=[early_stopping], verbose=2)
m1.save("models/shakespeare_gru_1.keras")

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_3_1/Cast:0", shape=(None, None), dtype=float32). Expected shape (None, None, 39), but input has incompatible shape (None, None)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, None), dtype=int64)
  • training=True
  • mask=None